## Slicing en Numpy

Pueden seguir esta notebook en Colab, haciendo clic abajo:

<a href="https://colab.research.google.com/github/lbiedma/an2famaf2022/blob/main/notebooks/numpy_slicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para este cuatrimestre, uno de nuestros objetivos es aprovechar el poder de algunas de las funcionalidades internas que trae la librería Numpy, que hacen más rápido el cálculo de las operaciones más básicas del álgebra lineal (sumas y multiplicaciones de matrices y vectores).

Si quieren conocer más sobre el tema, la necesidad de hacer álgebra lineal numérica rápida viene desde hace una buena cantidad de tiempo y LA librería por excelencia para hacer eso es [BLAS](http://www.netlib.org/blas/). Numpy provee su implementación de las funciones de esta librería, descargándola de forma automática cuando es instalada.

Si queremos aprovechar el poder de estas librerías, es necesario reducir la cantidad de bucles que usamos en nuestros cálculos y comenzar a realizar operaciones en bloques o rebanadas de matrices y vectores.

In [ ]:
# Como siempre, importemos la librería y definamos algunos elementos de ejemplo
import numpy as np
# Recordemos agregar el dtype si vamos a trabajar con flotantes
matriz = np.array(range(36), dtype=float).reshape((6, 6))
vector = np.array(range(6), dtype=float)

print("matriz = \n", matriz)
print("vector = \n", vector)

### Obtener todos los elementos del vector HASTA (sin contar a) N

In [ ]:
N = 4
print(vector[:N])

### Obtener todos los elementos del vector DESDE N

In [ ]:
print(vector[N:])

### Obtener todos los elementos del vector DESDE M HASTA (sin contar a) N

In [ ]:
M = 2
print(vector[M:N])

### Es posible usar índices negativos, lo cual puede ser de utilidad en algunos casos.
Por ejemplo si queremos llegar al anteúltimo elemento usamos '-1'

In [ ]:
print(vector[:-1])

### Obtener los elementos del vector al revés, o una parte al revés
El primer ':' separa entre principio y fin, el segundo ':' indica cómo avanzar

In [ ]:
print(vector[::-1])
print(vector[N:M:-1])

### Podemos obtener los elementos de acuerdo a un índice que definamos como una lista de Python

In [ ]:
indice = [2, 0, 3, 4]
print(vector[indice])

### Podemos hacer lo mismo con matrices, tomando pedazos a través de cualquiera de sus ejes [filas, columnas].
Recuerden que un índice desde 0 hasta 0 no nos va a dar un array vacío.

In [ ]:
for idx in range(7):
    print("matriz hasta {}".format(idx))
    print(matriz[:idx, :idx])

### La columna i-ésima de una matriz se consigue sin tocar la primera dimensión (usando ':')

In [ ]:
print(matriz[:, 3])
# NOTAR: Esto nos da un array de Numpy de una dimensión, no genera uno de dos dimensiones con sólo una columna.

### Nuevamente, podemos usar índices en todas las direcciones y combinarlos

In [ ]:
indice_filas = [2, 4]
print(matriz[np.array(indice_filas), :4])

## Operaciones
Veamos por qué es tan útil trabajar con las operaciones que vienen en las librerías de Numpy.

Definamos una matriz bastante grande formada por números aleatorios con una distribución normal (esta matriz es inversible con probabilidad 1, por qué?). Vamos a definir también un vector al que podamos multiplicar por ella.

In [ ]:
import time
# Si se pone muy lenta la máquina (problemas de memoria), pueden achicar un poco el número
big_N = 10000
big_matrix = np.random.random((big_N, big_N))
big_vector = np.random.random(big_N)

### Hagamos la multiplicación matriz - vector a manopla y veamos cuánto tiempo toma (vayan a armar el mate o algo...)

In [ ]:
# Generamos un vector vacío
resultado = np.empty(big_N)
start = time.time()
for fila in range(big_N):
    resultado[fila] = 0.0
    for columna in range(big_N):
        resultado[fila] += big_matrix[fila, columna] * big_vector[columna];

end = time.time()
print("La operación tomó {} segundos".format(end - start))

### Qué pasa cuando lo hacemos usando @?

In [ ]:
resultado = np.empty(big_N)
start = time.time()
resultado = big_matrix @ big_vector
end = time.time()
print("La operación tomó {} segundos".format(end - start))

### Quieren probar cuánto toma el producto matriz-matriz? Pueden ejecutar abajo, va a tardar demasiado tiempo
(lo que haya tardado la versión anterior multiplicado por big_N...)

In [ ]:
resultado = np.empty((big_N, big_N))
start = time.time()
for fila in range(big_N):
    for columna in range(big_N):
        resultado[fila, columna] = 0.0
        for idx in range(big_N):
                resultado[fila, columna] += big_matrix[fila, idx] * big_matrix[idx, columna]
end = time.time()
print("La operación tomó {} segundos".format(end - start))

In [ ]:
start = time.time()
resultado = big_matrix @ big_matrix
end = time.time()
print("La operación tomó {} segundos".format(end - start))

## Solución de sistemas lineales
Resolvamos un sistema triangular inferior también, usemos la función de Numérico I y luego slicing.

In [ ]:
def sol_trinf(A, b):
    n = A.shape[0]
    x = b.copy()
    
    for idx in range(n):
        for jdx in range(idx):
            x[idx] = x[idx] - A[idx, jdx] * x[jdx]
        x[idx] = x[idx] / A[idx, idx]

    return x

In [ ]:
big_N = 50
# Generamos una matriz random y la hacemos triangular inferior
big_matrix = np.tril(np.random.random((big_N, big_N)))
big_vector = np.random.random(big_N)
start = time.time()
x = sol_trinf(big_matrix, big_vector)
end = time.time()
print("La operación tomó {} segundos".format(end - start))
print("Estamos cerca de la solución? Norma de la resta: {}".format(np.linalg.norm(big_matrix @ x - big_vector)))

In [ ]:
# Probemos con slicing
def sol_trinffil(A, b):
    n = len(b)
    x = b.copy()
    for i in range(n):
        x[i] = (b[i] - A[i, :i]@x[:i])/A[i,i]
    return x

In [ ]:
big_matrix = np.tril(np.random.random((big_N, big_N)))
big_vector = np.random.random(big_N)
start = time.time()
x = sol_trinffil(big_matrix, big_vector)
end = time.time()
print("La operación tomó {} segundos".format(end - start))
print("Estamos cerca de la solución? Norma de la resta: {}".format(np.linalg.norm(big_matrix @ x - big_vector)))

## Usando Numpy para contrastar
Vamos a empezar a usar pedazos de matrices y vectores para hacer algunas de las operaciones.
Lo bueno es que Numpy (y/o Scipy) ya tiene su propia implementación de todos los métodos que vamos a usar este cuatrimestre, como las descomposiciones de Cholesky, LU o QR (Documentación en https://numpy.org/doc/stable/reference/routines.linalg.html). Podremos contrastar nuestro resultado con lo que dé ahí (no pretendan que nuestras funciones tarden menos que las de Numpy, porque usan [LAPACK](http://www.netlib.org/lapack/))